<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/develop/kaggle_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [0]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

In [0]:
#Unzipping if needed
!unzip creditcard.zip

In [0]:
#read file & check the upload
df = pd.read_csv('indicators_joined_csv')
df.head()

In [0]:
data_shape = df.shape
print(data_shape)

In [0]:
#data cleaning & exploration